# Algorithm 3: Relative Position Encoding (Boltz)

Encodes relative positions between tokens with chain and entity information.

## Source Code Location
- **File**: `Boltz-Ref-src/boltz-official/src/boltz/model/layers/relative.py`

In [ ]:
import numpy as np
np.random.seed(42)

def one_hot(indices, num_classes):
    return np.eye(num_classes)[np.clip(indices, 0, num_classes-1)]

In [ ]:
def relative_position_encoding(residue_index, chain_index, entity_index, 
                                max_rel_pos=32, max_rel_chain=2):
    """
    Compute relative position encoding.
    
    Args:
        residue_index: Residue indices [N]
        chain_index: Chain indices [N]
        entity_index: Entity indices [N]
        max_rel_pos: Maximum relative position
        max_rel_chain: Maximum relative chain
    
    Returns:
        Relative encoding [N, N, num_features]
    """
    N = len(residue_index)
    
    print(f"Relative Position Encoding")
    print(f"="*50)
    print(f"Tokens: {N}")
    
    # Relative residue position
    rel_pos = residue_index[:, None] - residue_index[None, :]
    rel_pos_clipped = np.clip(rel_pos, -max_rel_pos, max_rel_pos) + max_rel_pos
    rel_pos_onehot = one_hot(rel_pos_clipped, 2 * max_rel_pos + 1)
    
    # Same chain indicator
    same_chain = (chain_index[:, None] == chain_index[None, :]).astype(np.float32)
    
    # Same entity indicator
    same_entity = (entity_index[:, None] == entity_index[None, :]).astype(np.float32)
    
    # Relative chain
    rel_chain = chain_index[:, None] - chain_index[None, :]
    rel_chain_clipped = np.clip(rel_chain, -max_rel_chain, max_rel_chain) + max_rel_chain
    rel_chain_onehot = one_hot(rel_chain_clipped, 2 * max_rel_chain + 1)
    
    # Concatenate
    features = np.concatenate([
        rel_pos_onehot,      # [N, N, 65]
        same_chain[:, :, None],  # [N, N, 1]
        same_entity[:, :, None], # [N, N, 1]
        rel_chain_onehot,    # [N, N, 5]
    ], axis=-1)
    
    print(f"Features: {features.shape[-1]} dimensions")
    
    return features

In [ ]:
# Test
print("Test: Relative Position Encoding")
print("="*60)

# Simulate a 2-chain complex
chain1_len = 30
chain2_len = 20
N = chain1_len + chain2_len

residue_index = np.concatenate([np.arange(chain1_len), np.arange(chain2_len)])
chain_index = np.concatenate([np.zeros(chain1_len), np.ones(chain2_len)]).astype(np.int32)
entity_index = np.concatenate([np.zeros(chain1_len), np.ones(chain2_len)]).astype(np.int32)

features = relative_position_encoding(residue_index, chain_index, entity_index)

print(f"\nOutput shape: {features.shape}")
print(f"Same chain (0,0): {features[0, 0, -7]}")
print(f"Same chain (0,35): {features[0, 35, -7]}")

## Key Insights

1. **Relative Position**: One-hot encoded relative positions
2. **Chain Information**: Same-chain and relative-chain features
3. **Entity Information**: Distinguishes different molecular entities
4. **Multi-chain Support**: Designed for protein complexes and ligands